In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D 
from sklearn import linear_model
import seaborn as sns
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from pandas import DataFrame
import math
import numpy
from statistics import *
from numpy import nan

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Make suitable changes here. My .csv file is in :  My Drive/Colab Notebooks
from google.colab import drive
drive.mount("/content/drive/")
%cd '/content/drive/My Drive/DS440/Income_and_race'
!pwd #Prints the present working directory
racedata = pd.read_csv('race.csv')
racedata.head()

Mounted at /content/drive/
/content/drive/My Drive/DS440/Income_and_race
/content/drive/My Drive/DS440/Income_and_race


,census_block_group,total.population,total.population.ME,race.white.alone.estimate,race.white.alone.ME,african.american.alone.estimate,african.american.alone.ME,american.indian.alaskan.native.estimate,american.indian.alaskan.native.ME,asian.estimate,asian.ME,native.hawaiian.other.islander.estimate,native.hawaiian.other.islander.ME,other.race.alone,other.race.ME,two.or.more.estimate,two.or.more.ME,Two.races.excluding.Some.other.race.estimate,Two.races.excluding.Some.other.race.ME,Two.races.excluding.Some.other.race.and.three.or.more.races.estimate,Two.races.excluding.Some.other.race.and.three.or.more.races.ME
0,10010201001,745,226,585,198,160,140,0,11,0,11,0,11,0,11,0,11,0,11,0,11
1,10010201002,1265,232,1083,222,104,75,0,11,9,13,0,11,0,11,69,46,0,11,69,46
2,10010202001,960,261,361,179,568,195,0,11,0,11,13,20,0,11,18,30,0,11,18,30
3,10010202002,1236,227,615,154,571,168,0,11,24,36,0,11,10,12,16,24,0,11,16,24
4,10010203001,2364,378,1481,314,515,197,22,34,27,32,6,14,228,285,85,103,0,11,85,103


In [3]:
minorityportion = 1-(racedata['race.white.alone.estimate'].astype(float)/racedata['total.population'].astype(float))
minorityportion = minorityportion.astype(float)
racedata["minorityportion"]=minorityportion
racedata.head()

,census_block_group,total.population,total.population.ME,race.white.alone.estimate,race.white.alone.ME,african.american.alone.estimate,african.american.alone.ME,american.indian.alaskan.native.estimate,american.indian.alaskan.native.ME,asian.estimate,asian.ME,native.hawaiian.other.islander.estimate,native.hawaiian.other.islander.ME,other.race.alone,other.race.ME,two.or.more.estimate,two.or.more.ME,Two.races.excluding.Some.other.race.estimate,Two.races.excluding.Some.other.race.ME,Two.races.excluding.Some.other.race.and.three.or.more.races.estimate,Two.races.excluding.Some.other.race.and.three.or.more.races.ME,minorityportion
0,10010201001,745,226,585,198,160,140,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0.214765
1,10010201002,1265,232,1083,222,104,75,0,11,9,13,0,11,0,11,69,46,0,11,69,46,0.143874
2,10010202001,960,261,361,179,568,195,0,11,0,11,13,20,0,11,18,30,0,11,18,30,0.623958
3,10010202002,1236,227,615,154,571,168,0,11,24,36,0,11,10,12,16,24,0,11,16,24,0.502427
4,10010203001,2364,378,1481,314,515,197,22,34,27,32,6,14,228,285,85,103,0,11,85,103,0.373519


In [4]:
#racedata['census_block_group'] = racedata['census_block_group'].apply(lambda x: '{:.0f}'.format(x))
racedata.tail()


,census_block_group,total.population,total.population.ME,race.white.alone.estimate,race.white.alone.ME,african.american.alone.estimate,african.american.alone.ME,american.indian.alaskan.native.estimate,american.indian.alaskan.native.ME,asian.estimate,asian.ME,native.hawaiian.other.islander.estimate,native.hawaiian.other.islander.ME,other.race.alone,other.race.ME,two.or.more.estimate,two.or.more.ME,Two.races.excluding.Some.other.race.estimate,Two.races.excluding.Some.other.race.ME,Two.races.excluding.Some.other.race.and.three.or.more.races.estimate,Two.races.excluding.Some.other.race.and.three.or.more.races.ME,minorityportion
220328,721537506011,921,332,752,348,0,13,0,13,0,13,0,13,82,60,87,130,0,13,87,130,0.183496
220329,721537506012,2703,531,2230,459,207,175,40,63,0,13,0,13,200,192,26,39,26,39,0,13,0.174991
220330,721537506013,1195,425,1085,429,25,31,0,13,0,13,0,13,42,67,43,67,0,13,43,67,0.092050
220331,721537506021,2005,471,1662,463,68,64,0,13,0,13,0,13,266,202,9,16,0,13,9,16,0.171072
220332,721537506022,736,371,588,280,0,13,0,13,0,13,0,13,105,157,43,48,0,13,43,48,0.201087


In [5]:
racedata.shape

(220333, 22)

In [6]:
# Make suitable changes here. My .csv file is in :  My Drive/Colab Notebooks
!pwd #Prints the present working directory
incomedata = pd.read_csv('income.csv')
incomedata.head()

/content/drive/My Drive/DS440/Income_and_race


,census_block_group,total.households,households.ME,less.than.10000,less.than.1000.ME,10000.to.14999,10000.to.14999.ME,15000.to.19999,15000.to.19999.ME,20000.to.24999,20000.to.24999.ME,25000.to.29999,25000.to.29999.ME,30000.to.34999,30000.to.34999.ME,35000.to.39999,35000.to.39999.ME,40000.to.44999,40000.to.44999.ME,45000.to.49999,45000.to.49999.ME,50000.to.59999,50000.to.59999.ME,60000.to.74999,60000.to.74999.ME,75000.to.99999,75000.to.99999.ME,10000.to.124999,10000.to.124999.ME,125000.to.149999,125000.to.149999.ME,150000.to.199999,150000.to.199999.ME,200000.or.more,200000.or.more.ME,MEDIAN.INCOME,MEDIAN.INCOME.ME
0,10010201001,284,53,24,29,33,25,23,22,25,20,39,49,5,9,0,11,10,11,0,11,12,12,43,31,35,32,6,9,29,27,0,11,0,11,NaN,NaN
1,10010201002,456,64,8,10,15,24,14,22,5,8,10,11,31,30,25,27,8,9,19,16,25,28,57,32,101,50,47,26,27,23,55,31,9,11,77813.0,11618.0
2,10010202001,386,78,47,39,108,47,37,31,0,11,23,27,4,9,13,13,20,27,0,11,4,7,28,21,89,47,9,13,4,7,0,11,0,11,25179.0,20248.0
3,10010202002,452,85,32,27,37,30,15,15,18,19,12,16,28,33,14,16,69,71,43,28,25,22,16,17,44,34,56,30,43,28,0,11,0,11,45104.0,6377.0
4,10010203001,824,108,23,24,34,50,16,18,17,19,47,38,44,39,51,45,33,29,26,29,203,96,121,55,82,50,87,48,18,21,22,20,0,11,55222.0,2034.0


In [7]:
missing_income_data=incomedata[incomedata["MEDIAN.INCOME"].isna()]
missing_income_data=missing_income_data[["census_block_group", "total.households", "less.than.10000", "10000.to.14999", "15000.to.19999", "20000.to.24999", "25000.to.29999", "30000.to.34999"
, "35000.to.39999", "40000.to.44999", "45000.to.49999", "50000.to.59999", "60000.to.74999", "75000.to.99999", "10000.to.124999", "125000.to.149999", "150000.to.199999", "200000.or.more"]]
missing_income_data = missing_income_data[missing_income_data['total.households'] != 0]
missing_income_data.head()
median=(missing_income_data["total.households"]+1)/2
flooring=numpy.floor(median)
missing_income_data["median"]=flooring
missing_income_data_temp=missing_income_data.drop(columns=["census_block_group", "total.households", "median"])
the_list=(missing_income_data_temp.values.tolist())
the_cum_list=[]
for i in range(0, len(the_list)):
  a_list=the_list[i]
  x=np.cumsum(a_list)
  the_cum_list.append(list(x))
missing_income_data.head()


#type(missing_income_data["the_list"])


,census_block_group,total.households,less.than.10000,10000.to.14999,15000.to.19999,20000.to.24999,25000.to.29999,30000.to.34999,35000.to.39999,40000.to.44999,45000.to.49999,50000.to.59999,60000.to.74999,75000.to.99999,10000.to.124999,125000.to.149999,150000.to.199999,200000.or.more,median
0,10010201001,284,24,33,23,25,39,5,0,10,0,12,43,35,6,29,0,0,142.0
88,10030112024,326,0,21,10,27,0,61,0,59,0,11,0,43,0,12,0,82,163.0
104,10030114062,318,0,42,52,25,16,19,9,10,9,9,0,15,64,0,40,8,159.0
283,10150020005,430,13,35,83,15,13,0,33,24,0,0,28,96,90,0,0,0,215.0
313,10159819011,4,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,2.0


In [8]:
pos=[]
check=[]
flooring_interest=list(flooring)

for i in range(0, len(the_cum_list)):
  #print(i)
  a_list=the_cum_list[i]
  interest=flooring_interest[i]
  #print(a_list)
  for n in range(0, len(a_list)):
    if (a_list[n]) < interest:
      continue
    else:
      #print(a_list)
      pos.append(n)
      check.append(a_list)
      break
len(check)




5289

In [9]:
pred_median=[]
for i in range(0, len(pos)):
  x=pos[i]
  if x==1:
    y=9999/2
  if x==2:
    y=(10000+ 14999)/2
  if x==3:
    y=(15000+ 19999)/2
  if x==4:
    y=(20000+ 24999)/2
  if x==5:
    y=(25000+ 29999)/2
  if x==6:
    y=(30000+ 34999)/2
  if x==7:
    y=(35000+ 39999)/2
  if x==8:
    y=(40000+ 44999)/2
  if x==9:
    y=(45000+49999)/2
  if x==10:
    y=(50000+ 59999)/2
  if x==11:
    y=(60000+ 74999)/2
  if x==12:
    y=(75000+ 99999)/2
  if x==13:
    y=(100000+ 124999)/2
  if x==14: 
    y=(125000+ 149999)/2
  if x==15:
    y=(150000+ 199999)/2
  if x==16:
    y=200000
  pred_median.append(y)
len(pred_median)
missing_income_data["MEDIAN.INCOME.PRED"]=pred_median
missing_income_data.head()

,census_block_group,total.households,less.than.10000,10000.to.14999,15000.to.19999,20000.to.24999,25000.to.29999,30000.to.34999,35000.to.39999,40000.to.44999,45000.to.49999,50000.to.59999,60000.to.74999,75000.to.99999,10000.to.124999,125000.to.149999,150000.to.199999,200000.or.more,median,MEDIAN.INCOME.PRED
0,10010201001,284,24,33,23,25,39,5,0,10,0,12,43,35,6,29,0,0,142.0,22499.5
88,10030112024,326,0,21,10,27,0,61,0,59,0,11,0,43,0,12,0,82,163.0,37499.5
104,10030114062,318,0,42,52,25,16,19,9,10,9,9,0,15,64,0,40,8,159.0,32499.5
283,10150020005,430,13,35,83,15,13,0,33,24,0,0,28,96,90,0,0,0,215.0,37499.5
313,10159819011,4,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,2.0,54999.5


In [10]:
missing_income_data_total = pd.merge(incomedata, missing_income_data, how="left", on=["census_block_group"])
missing_income_data_total.shape

(220333, 56)

In [11]:
final_list=[]
x=list(incomedata["MEDIAN.INCOME"])
x = [str(l) for l in x]
y=list(missing_income_data_total["MEDIAN.INCOME.PRED"])
y = [str(l) for l in y]

for i in range(0,len(x)):
  income=x[i]
  if income==nan or income=="nan":
    median=y[i]
    final_list.append(median)
  else:
    median=income
    final_list.append(median)

In [12]:
final_list = [float(l) for l in final_list]

In [13]:
final_list[:10]

[22499.5,
 77813.0,
 25179.0,
 45104.0,
 55222.0,
 27212.0,
 61806.0,
 45962.0,
 53095.0,
 63542.0]

In [14]:
incomedata["MEDIAN.INCOME"]=final_list

In [15]:
incomedata.head()

,census_block_group,total.households,households.ME,less.than.10000,less.than.1000.ME,10000.to.14999,10000.to.14999.ME,15000.to.19999,15000.to.19999.ME,20000.to.24999,20000.to.24999.ME,25000.to.29999,25000.to.29999.ME,30000.to.34999,30000.to.34999.ME,35000.to.39999,35000.to.39999.ME,40000.to.44999,40000.to.44999.ME,45000.to.49999,45000.to.49999.ME,50000.to.59999,50000.to.59999.ME,60000.to.74999,60000.to.74999.ME,75000.to.99999,75000.to.99999.ME,10000.to.124999,10000.to.124999.ME,125000.to.149999,125000.to.149999.ME,150000.to.199999,150000.to.199999.ME,200000.or.more,200000.or.more.ME,MEDIAN.INCOME,MEDIAN.INCOME.ME
0,10010201001,284,53,24,29,33,25,23,22,25,20,39,49,5,9,0,11,10,11,0,11,12,12,43,31,35,32,6,9,29,27,0,11,0,11,22499.5,NaN
1,10010201002,456,64,8,10,15,24,14,22,5,8,10,11,31,30,25,27,8,9,19,16,25,28,57,32,101,50,47,26,27,23,55,31,9,11,77813.0,11618.0
2,10010202001,386,78,47,39,108,47,37,31,0,11,23,27,4,9,13,13,20,27,0,11,4,7,28,21,89,47,9,13,4,7,0,11,0,11,25179.0,20248.0
3,10010202002,452,85,32,27,37,30,15,15,18,19,12,16,28,33,14,16,69,71,43,28,25,22,16,17,44,34,56,30,43,28,0,11,0,11,45104.0,6377.0
4,10010203001,824,108,23,24,34,50,16,18,17,19,47,38,44,39,51,45,33,29,26,29,203,96,121,55,82,50,87,48,18,21,22,20,0,11,55222.0,2034.0


In [16]:
result = pd.merge(incomedata, racedata, how="inner", on=["census_block_group"])
result = result[result['total.population'] != 0]

In [17]:
result.shape

(219227, 58)

In [18]:
result.head()

,census_block_group,total.households,households.ME,less.than.10000,less.than.1000.ME,10000.to.14999,10000.to.14999.ME,15000.to.19999,15000.to.19999.ME,20000.to.24999,20000.to.24999.ME,25000.to.29999,25000.to.29999.ME,30000.to.34999,30000.to.34999.ME,35000.to.39999,35000.to.39999.ME,40000.to.44999,40000.to.44999.ME,45000.to.49999,45000.to.49999.ME,50000.to.59999,50000.to.59999.ME,60000.to.74999,60000.to.74999.ME,75000.to.99999,75000.to.99999.ME,10000.to.124999,10000.to.124999.ME,125000.to.149999,125000.to.149999.ME,150000.to.199999,150000.to.199999.ME,200000.or.more,200000.or.more.ME,MEDIAN.INCOME,MEDIAN.INCOME.ME,total.population,total.population.ME,race.white.alone.estimate,race.white.alone.ME,african.american.alone.estimate,african.american.alone.ME,american.indian.alaskan.native.estimate,american.indian.alaskan.native.ME,asian.estimate,asian.ME,native.hawaiian.other.islander.estimate,native.hawaiian.other.islander.ME,other.race.alone,other.race.ME,two.or.more.estimate,two.or.more.ME,Two.races.excluding.Some.other.race.estimate,Two.races.excluding.Some.other.race.ME,Two.races.excluding.Some.other.race.and.three.or.more.races.estimate,Two.races.excluding.Some.other.race.and.three.or.more.races.ME,minorityportion
0,10010201001,284,53,24,29,33,25,23,22,25,20,39,49,5,9,0,11,10,11,0,11,12,12,43,31,35,32,6,9,29,27,0,11,0,11,22499.5,NaN,745,226,585,198,160,140,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0.214765
1,10010201002,456,64,8,10,15,24,14,22,5,8,10,11,31,30,25,27,8,9,19,16,25,28,57,32,101,50,47,26,27,23,55,31,9,11,77813.0,11618.0,1265,232,1083,222,104,75,0,11,9,13,0,11,0,11,69,46,0,11,69,46,0.143874
2,10010202001,386,78,47,39,108,47,37,31,0,11,23,27,4,9,13,13,20,27,0,11,4,7,28,21,89,47,9,13,4,7,0,11,0,11,25179.0,20248.0,960,261,361,179,568,195,0,11,0,11,13,20,0,11,18,30,0,11,18,30,0.623958
3,10010202002,452,85,32,27,37,30,15,15,18,19,12,16,28,33,14,16,69,71,43,28,25,22,16,17,44,34,56,30,43,28,0,11,0,11,45104.0,6377.0,1236,227,615,154,571,168,0,11,24,36,0,11,10,12,16,24,0,11,16,24,0.502427
4,10010203001,824,108,23,24,34,50,16,18,17,19,47,38,44,39,51,45,33,29,26,29,203,96,121,55,82,50,87,48,18,21,22,20,0,11,55222.0,2034.0,2364,378,1481,314,515,197,22,34,27,32,6,14,228,285,85,103,0,11,85,103,0.373519


In [19]:
result[result['MEDIAN.INCOME'].isna()]

,census_block_group,total.households,households.ME,less.than.10000,less.than.1000.ME,10000.to.14999,10000.to.14999.ME,15000.to.19999,15000.to.19999.ME,20000.to.24999,20000.to.24999.ME,25000.to.29999,25000.to.29999.ME,30000.to.34999,30000.to.34999.ME,35000.to.39999,35000.to.39999.ME,40000.to.44999,40000.to.44999.ME,45000.to.49999,45000.to.49999.ME,50000.to.59999,50000.to.59999.ME,60000.to.74999,60000.to.74999.ME,75000.to.99999,75000.to.99999.ME,10000.to.124999,10000.to.124999.ME,125000.to.149999,125000.to.149999.ME,150000.to.199999,150000.to.199999.ME,200000.or.more,200000.or.more.ME,MEDIAN.INCOME,MEDIAN.INCOME.ME,total.population,total.population.ME,race.white.alone.estimate,race.white.alone.ME,african.american.alone.estimate,african.american.alone.ME,american.indian.alaskan.native.estimate,american.indian.alaskan.native.ME,asian.estimate,asian.ME,native.hawaiian.other.islander.estimate,native.hawaiian.other.islander.ME,other.race.alone,other.race.ME,two.or.more.estimate,two.or.more.ME,Two.races.excluding.Some.other.race.estimate,Two.races.excluding.Some.other.race.ME,Two.races.excluding.Some.other.race.and.three.or.more.races.estimate,Two.races.excluding.Some.other.race.and.three.or.more.races.ME,minorityportion
2408,10970036051,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,0,11,NaN,NaN,653,370,381,219,207,142,0,11,9,11,0,11,13,17,43,36,0,11,43,36,0.416539
3468,20200006001,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,NaN,NaN,185,175,109,106,0,9,66,77,0,9,0,9,3,6,7,11,0,9,7,11,0.410811
3671,20600001002,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,NaN,NaN,15,8,10,8,3,3,0,9,0,9,0,9,2,5,0,9,0,9,0,9,0.333333
3905,21850003001,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,NaN,NaN,2892,412,2416,355,32,25,214,88,61,39,11,16,50,36,108,69,0,9,108,69,0.164592
4486,40130610171,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,0,12,NaN,NaN,3389,1003,2210,564,398,200,207,108,0,12,4,9,232,132,338,187,19,34,319,186,0.347890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213064,550090211001,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,NaN,NaN,1060,190,413,100,528,142,92,50,20,31,0,9,7,13,0,9,0,9,0,9,0.610377
213360,550250011022,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,NaN,NaN,2790,226,2169,225,41,33,8,15,432,148,0,9,6,12,134,56,16,24,118,55,0.222581
213394,550250016064,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,9,NaN,NaN,2411,181,2112,191,85,53,9,18,102,71,0,9,9,18,94,79,8,16,86,78,0.124015
218547,720371601001,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,NaN,NaN,4,9,4,9,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0.000000


In [20]:
result.shape

(219227, 58)

In [21]:
relevant_data = result[["census_block_group", "minorityportion", "MEDIAN.INCOME", "total.population"]]
relevant_data.head()

,census_block_group,minorityportion,MEDIAN.INCOME,total.population
0,10010201001,0.214765,22499.5,745
1,10010201002,0.143874,77813.0,1265
2,10010202001,0.623958,25179.0,960
3,10010202002,0.502427,45104.0,1236
4,10010203001,0.373519,55222.0,2364


In [22]:
type(relevant_data["total.population"])

pandas.core.series.Series

In [23]:
population=relevant_data["total.population"]
census_block_classification=[]
len(list(population))

219227

In [24]:
def census_classification(population):
  for i in range(0,len(population)):
    int_population = (list(population)[i])
    if int_population<2500:
      classification="Rural"
      census_block_classification.append(classification)
    elif int_population>=50000:
      classification="Urbanized Area"
      census_block_classification.append(classification)
    else:
      classification="Urban Clusters"
      census_block_classification.append(classification)
  return census_block_classification



In [25]:

census_block_classification=census_classification(population)

In [26]:
relevant_data["census_block_classification"]=census_block_classification

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
relevant_data

,census_block_group,minorityportion,MEDIAN.INCOME,total.population,census_block_classification
0,10010201001,0.214765,22499.5,745,Rural
1,10010201002,0.143874,77813.0,1265,Rural
2,10010202001,0.623958,25179.0,960,Rural
3,10010202002,0.502427,45104.0,1236,Rural
4,10010203001,0.373519,55222.0,2364,Rural
...,...,...,...,...,...
220328,721537506011,0.183496,13171.0,921,Rural
220329,721537506012,0.174991,24308.0,2703,Urban Clusters
220330,721537506013,0.092050,40625.0,1195,Rural
220331,721537506021,0.171072,9006.0,2005,Rural


In [28]:
relevant_data.to_csv('relevant.data.csv')  

In [30]:
result["census_block_classification"]=census_block_classification

In [31]:
result.to_csv('interesting_data.csv')